In [ ]:
!pip install --q openai langchain langchainhub tiktoken chromadb
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# OpenAI API 키 설정

# 1. 웹 데이터 로드
web_loader = WebBaseLoader([
    "https://www.yes24.com/Product/Goods/128917300",
    "https://www.yes24.com/Product/Goods/128944230",
    "https://www.yes24.com/Product/Goods/128266166"
])
data = web_loader.load()

# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)
all_splits = text_splitter.split_documents(data)

# 3. 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
)

# 4. 프롬프트 템플릿 설정
prompt = ChatPromptTemplate(input_variables=['context', 'question'],
                            messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(
                                input_variables=['context', 'question'],
                                template=
                                '''You are an assistant for question-answering tasks.
                                Use the following pieces of retrieved context to answer the question.
                                If you don't know the answer, just say that you don't know.
                                Use three sentences maximum and keep the answer concise.
                                \nQuestion: {question}
                                \nContext: {context}
                                \nAnswer:'''
                            ))])

# 5. 모델 설정
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)

# 6. 질의 응답 체인 설정
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

# 7. 질문에 대한 응답 생성 함수
def get_chat_response(question):
    response = qa_chain({"query": question})
    return response["result"]

# 사용자 입력을 받아 질문에 대한 응답을 생성하는 부분
if __name__ == "__main__":
    while True:
        user_question = input("질문을 입력하세요: ")
        if user_question.lower() in ['exit', 'quit', 'q']:
            break
        result = get_chat_response(user_question)
        print(f"응답: {result}")


응답: 《다시, 역사의 쓸모》는 대한민국 대표 역사 커뮤니케이터 최태성이 지난 5년간 새롭게 발굴한 역사의 쓸모를 담은 책으로, '역사를 배워야 하는 이유'에 답했던 전작에서 한 걸음 더 나아가 '삶에 역사의 지혜를 들여오는 방법'을 다루고 있습니다. 최태성은 역사 속 인물들의 이야기를 현대인의 고민과 현대 사회의 문제에 자유자재로 엮어 한 편의 이야기로 완성하며, 독자들을 역사의 쓸모 세계로 초대합니다. 이 책은 역사를 통해 현재를 바라보는 건강한 시선을 점검하는 일에 초점을 맞추고 있습니다.
